# 4.1 多层感知机

## 激活函数

+ Relu：$ReLu(x)=max(x,0)$<div align=center><img src=./img/chapter04/1.png width=50%></div>
+ sigmoid:$sigmoid(x)=\frac{1}{1+exp(-x)}$<div align=center><img src=./img/chapter04/2.png width=50%></div>
+ tanh:$tanh(x)=\frac{1-exp(-2x)}{1+exp(-2x)}$ <div align=center><img src=./img/chapter04/3.png width=50%></div>
+ Gelu:$GeLu(x)=0.5x(1+tanh(\sqrt{2/\pi}(x+0.044715x^3)))$<div align=center><img src=./img/chapter04/4.png width=50%></div>

# 4.2 多层感知机简洁实现

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

net=nn.Sequential(
    nn.Flatten(),
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10)
)
def init_weights(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
net.apply(init_weights)
batch_size, lr, num_epochs = 256, 0.1, 10 
loss = nn.CrossEntropyLoss() 
trainer = torch.optim.SGD(net.parameters(), lr=lr)
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size) 
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

# 4.4 模型选择、欠拟合和过拟合
什么是过拟合？

将模型在训练数据上拟合得比在潜在分布中更接近的现象称为过拟合(overfitting)，用于对抗过拟合的技术称为正则化(regularization)。

如何判断过拟合和欠拟合？

当训练准确率和测试准确率相差过大，可以判断为过拟合。

当训练准确率和测试准确率相差甚小，且训练准确率未达到最优。
<div align=center>
<img src=./img/chapter04/5.png width=50%>
</div>

# 4.5正则化技术
正则化（Regularization）是一类通过限制模型复杂度，从而避免过拟合，提 高泛化能力的方法，比如引入约束、增加先验、提前停止等
## 4.5.1 L1和L2正则化
原有的损失函数：$L(w,b)=\frac{1}{2n}\sum_{i=1}^{n}(w^Tx^{(i)}+b-y^{(i)})^2$

加入L2正则化后：$L(w,b)+\frac{\lambda}{2}||w||^2$,而L1正则化则是取绝对值
## 4.5.2 权重衰退
在每次参数更新时，引入一个衰减系数：$\theta_t\leftarrow(1-\beta)\theta_{t-1}-\alpha g_t$

其中$g_t$是第t布更新时的梯度，$\alpha$为学习率，$\beta$为权重衰退系数。在标准的随机梯度下降中，权重衰减正则化和L2正则化的效 果相同．因此，权重衰减在一些深度学习框架中通过 L2正则化来实现。但是，在较为复杂的优化方法（比如 Adam）中，权重衰减正则化和L2正则化并不等价。

### 4.5.3 提前停止
常见操作，当验证集准确率收敛时，停止迭代
### 4.5.4 丢弃法
当训练一个深度神经网络时， 我们可以随机丢弃一部分神经元（同时丢 弃其对应的连接边）来避免过拟合，这种方法称为丢弃法每次选择丢弃的神经元是随机的．最简单的方法是 设置一个固定的概率 𝑝．对每一个神经元都以概率 𝑝 来判定要不要保留．对于 一个神经层 𝒚 = 𝑓(𝑾 𝒙 + 𝒃)，我们可以引入一个掩蔽函数mask(⋅) 使得 𝒚 = 𝑓(𝑾 mask(𝒙) + 𝒃)．掩蔽函数 mask(⋅) 的定义为
<div align=center><img src=./img/chapter04/6.png></div>

其中$m\in {0,1}^D$是丢弃掩码。通过以概率为𝑝的伯努利分布随机生成．在训练时，激活神经元的平均数量为原来的𝑝倍．而在测试时，所有的神经元都是可以激活的，这会造成训练和测试时网络的输出不一致．为了缓解这个问题，在测试时需要将神经层的输入𝒙乘以𝑝，也相当于把不同的神经网络做了平均．保留率𝑝可以通过验证集来选取一个最优的值．
<div align=center><img src=./img/chapter04/7.png></div>

### 4.5.5 数据增强
在数据量有限的情况下，通过数据增强增加数据量，提高模型鲁棒性，避免过拟合。
+ 旋转(Rotation)：将图像按顺时针或逆时针方向随机旋转一定角度．
+ 翻转(Flip)：将图像沿水平或垂直方向随机翻转一定角度
+ 缩放（Zoom In/Out）：将图像放大或缩小一定比例
+ 平移（Shift）：将图像沿水平或垂直方法平移一定步长
+ 加噪声（Noise）：加入随机噪声
### 4.5.6 标签平滑
在输出标签中添加噪声来避免 模型过拟合